In [ ]:
#!pip3 install s3fs --user

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
import findspark
findspark.init()

In [36]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [37]:
# 데이터를 hdfs에서 가지고 옴
train_data = spark.read.csv("hdfs:///user/ubuntu/dataym/2009/02/02.csv", )
# test_data = spark.read.csv("")

In [38]:
train_data.show(5)

+--------------------+----+--------------------+----------+---------+----------+---------+---+
|                 _c0| _c1|                 _c2|       _c3|      _c4|       _c5|      _c6|_c7|
+--------------------+----+--------------------+----------+---------+----------+---------+---+
|2009-02-27 13:31:...|16.5|2009-02-27 13:31:...|-73.974938|40.762091|-74.003309|40.706195|  2|
|2009-02-13 16:54:...| 6.7|2009-02-13 16:54:...|-73.959669|40.779718|-73.976259|40.788413|  1|
|2009-02-22 22:48:...| 2.5|2009-02-22 22:48:...|         0|        0|-74.005433|40.726685|  2|
|2009-02-16 07:40:...|14.5|2009-02-16 07:40:...|-73.948872|40.807601|-73.937218|40.767366|  1|
|2009-02-27 17:39:...| 8.1|2009-02-27 17:39:...|-73.992877|40.742668|-73.983468| 40.74748|  1|
+--------------------+----+--------------------+----------+---------+----------+---------+---+
only showing top 5 rows



In [39]:
# test_data.show(5)

In [40]:
type(train_data)

pyspark.sql.dataframe.DataFrame

In [41]:
# train_data가 너무 크니 10000개로 자르기
little_train_data = train_data.limit(10000)

In [42]:
# pyspark.sql.dataframe.DataFrame to Pandas df
train_df = little_train_data.toPandas()
# test_df = test_data.toPandas()
display(train_df.head(2))
# test_df.head(2)

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7
0,2009-02-27 13:31:21.0000003,16.5,2009-02-27 13:31:21 UTC,-73.974938,40.762091,-74.003309,40.706195,2
1,2009-02-13 16:54:52.0000006,6.7,2009-02-13 16:54:52 UTC,-73.959669,40.779718,-73.976259,40.788413,1


In [43]:
train_column_list =  ["key","fare_amount","pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count"]
test_column_list = ["key","pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count"]

In [44]:
train_df.columns = train_column_list
# test_df.columns = test_column_list

In [45]:
train_df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-02-27 13:31:21.0000003,16.5,2009-02-27 13:31:21 UTC,-73.974938,40.762091,-74.003309,40.706195,2
1,2009-02-13 16:54:52.0000006,6.7,2009-02-13 16:54:52 UTC,-73.959669,40.779718,-73.976259,40.788413,1
2,2009-02-22 22:48:00.000000130,2.5,2009-02-22 22:48:00 UTC,0,0,-74.005433,40.726685,2
3,2009-02-16 07:40:14.0000002,14.5,2009-02-16 07:40:14 UTC,-73.948872,40.807601,-73.937218,40.767366,1
4,2009-02-27 17:39:00.000000125,8.1,2009-02-27 17:39:00 UTC,-73.992877,40.742668,-73.983468,40.74748,1
...,...,...,...,...,...,...,...,...
9995,2009-02-24 18:23:00.000000169,6.5,2009-02-24 18:23:00 UTC,-73.961238,40.75665,-73.976258,40.760223,1
9996,2009-02-27 08:16:00.0000003,11.3,2009-02-27 08:16:00 UTC,-74.006757,40.730563,-73.973948,40.751045,5
9997,2009-02-27 08:44:26.0000004,6.5,2009-02-27 08:44:26 UTC,-73.973492,40.753211,-73.984299,40.742373,1
9998,2009-02-24 08:40:07.0000002,12.5,2009-02-24 08:40:07 UTC,-73.960046,40.770377,-73.981899,40.752158,1


In [46]:
train_df.describe()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,10000,10000,10000,10000,10000,10000,10000,10000
unique,10000,349,8726,8839,9103,9014,9235,6
top,2009-02-27 13:31:21.0000003,4.9,2009-02-06 18:20:00 UTC,0,0,0,0,1
freq,1,441,5,154,154,151,151,6849


In [51]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                10000 non-null  datetime64[ns]     
 1   fare_amount        10000 non-null  float64            
 2   pickup_datetime    10000 non-null  datetime64[ns, UTC]
 3   pickup_longitude   10000 non-null  float64            
 4   pickup_latitude    10000 non-null  float64            
 5   dropoff_longitude  10000 non-null  float64            
 6   dropoff_latitude   10000 non-null  float64            
 7   passenger_count    10000 non-null  int64              
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(5), int64(1)
memory usage: 625.1 KB


In [47]:
train_df['key'] = pd.to_datetime(train_df['key'])
train_df['pickup_datetime'] = pd.to_datetime(train_df['pickup_datetime'])
# test_df['key'] = pd.to_datetime(test_df['key'])
# test_df['pickup_datetime']  = pd.to_datetime(test_df['pickup_datetime'])

In [49]:
train_df = train_df.astype({'fare_amount':'float',
              'pickup_longitude':'float',
              'pickup_latitude':'float',
              'dropoff_longitude':'float',
              'dropoff_latitude':'float',
              'passenger_count':'int'},)
# train_df = train_df.astype({'pickup_longitude':'float',
#               'pickup_latitude':'float',
#               'dropoff_longitude':'float',
#               'dropoff_latitude':'float',
#               'passenger_count':'int'},)

In [50]:
#Dtype이 object를 float,int로 변경
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                10000 non-null  datetime64[ns]     
 1   fare_amount        10000 non-null  float64            
 2   pickup_datetime    10000 non-null  datetime64[ns, UTC]
 3   pickup_longitude   10000 non-null  float64            
 4   pickup_latitude    10000 non-null  float64            
 5   dropoff_longitude  10000 non-null  float64            
 6   dropoff_latitude   10000 non-null  float64            
 7   passenger_count    10000 non-null  int64              
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(5), int64(1)
memory usage: 625.1 KB


In [52]:
train_df.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [ ]:
#하버 사인 (haversine)  공식
pip install haversine

In [53]:
train_df['fare_amount'].describe()

count    10000.000000
mean         9.766464
std          7.641671
min          2.500000
25%          5.700000
50%          7.700000
75%         10.900000
max        146.900000
Name: fare_amount, dtype: float64

In [54]:
train_df.drop(train_df[train_df['pickup_longitude'] == 0].index, axis=0, inplace = True)
train_df.drop(train_df[train_df['pickup_latitude'] == 0].index, axis=0, inplace = True)
train_df.drop(train_df[train_df['dropoff_longitude'] == 0].index, axis=0, inplace = True)
train_df.drop(train_df[train_df['dropoff_latitude'] == 0].index, axis=0, inplace = True)
train_df.drop(train_df[train_df['passenger_count'] == 208].index, axis=0, inplace = True)
train_df.drop(train_df[train_df['passenger_count'] > 5].index, axis=0, inplace = True)
train_df.drop(train_df[train_df['passenger_count'] == 0].index, axis=0, inplace = True)

In [57]:
train_df.head(2)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-02-27 13:31:21.000000300,16.5,2009-02-27 13:31:21+00:00,-73.974938,40.762091,-74.003309,40.706195,2
1,2009-02-13 16:54:52.000000600,6.7,2009-02-13 16:54:52+00:00,-73.959669,40.779718,-73.976259,40.788413,1


In [59]:
train_df.drop(['key'], axis=1,inplace=True)

In [60]:
train_df.drop(['pickup_datetime'], axis=1,inplace=True)

In [61]:
train_df.head(2)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,16.5,-73.974938,40.762091,-74.003309,40.706195,2
1,6.7,-73.959669,40.779718,-73.976259,40.788413,1


In [63]:
train_df.dropna(inplace=True)

train_df.drop(train_df.index[(train_df.pickup_longitude < -75) | 
           (train_df.pickup_longitude > -72) | 
           (train_df.pickup_latitude < 40) | 
           (train_df.pickup_latitude > 42)],inplace=True)
train_df.drop(train_df.index[(train_df.dropoff_longitude < -75) | 
           (train_df.dropoff_longitude > -72) | 
           (train_df.dropoff_latitude < 40) | 
           (train_df.dropoff_latitude > 42)],inplace=True)

In [ ]:
train_df.head()

In [ ]:
# test_df.drop(['key'], axis=1,inplace=True)

In [ ]:
# test_df.head()

In [ ]:
# test_df.dropna(inplace=True)

# test_df.drop(test_df.index[(test_df.pickup_longitude < -75) | 
#            (test_df.pickup_longitude > -72) | 
#            (test_df.pickup_latitude < 40) | 
#            (test_df.pickup_latitude > 42)],inplace=True)
# test_df.drop(test_df.index[(test_df.dropoff_longitude < -75) | 
#            (test_df.dropoff_longitude > -72) | 
#            (test_df.dropoff_latitude < 40) | 
#            (test_df.dropoff_latitude > 42)],inplace=True)

In [ ]:
# test_df.head()

In [ ]:
# test_df.drop(['pickup_datetime'], axis=1,inplace=True)

In [ ]:
# df.to_csv('s3://team07-data/SPARK-ETL/pyspark_ETL.csv', index=False)
train_df.to_csv('', index=False)
# test_df.to_csv('', index-False)